# Instalo e importo librerías

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=e7650a9ab286c879f79c8235d7f2faaa53bbd00dbfa169af63a7ffc5cc70809d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Creo sesión de spark y leo csv

In [ ]:
downloaded = drive.CreateFile({'id':"10xgOf2rORcGlcKPME3cHGHssXGzNemer"})
downloaded.GetContentFile('GooglePlayStore_User_Reviews.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('GooglePlayStore_User_Reviews.csv', header=True, inferSchema=True)
user_reviews = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Ejercicio 11

Realizar un análisis de stopwords de las reviews. Dada la frecuencia de los tokens de las reviews, mostrar los 30 tokens más frecuentes y listar del total de tokens cuales son stopwords utilizando nltk. (⭐⭐)

**ACLARACION:** En slack dijeron que hay que listar las stopwords de los 30 tokens más frecuentes y no del TOTAL, como dice en el enunciado. Se entregó también la otra versión (listando las stopwords del total de tokens) en otro notebook.

Filtro las reviews nan ya que las considero un error del archivo (y además no me sirven para analizar las palabras de cada una).

Uso distinct para eliminar reviews que se encuentran repetidas (provoca que se cuenten más de una vez palabras o tokens).

Mapeo para quedarme solo con las reviews.

Uso flatMap para quedarme con todos los tokens de cada review (habrá repeticiones por usar flatMap). Como tokenizer utilizo el de nltk.

Mapeo para quedarme los tokens (todos en minúscula) como clave y 1 como valor, para desppués sumarlos con reduceByKey, quedándome así con todos los tokens y sus frecuencias.

Cacheo porque haré más de una acción sobre esto (tomo los 30 más frecuentes ordenándolos y luego filtro por stopwords y por estos 30 tokens).


In [ ]:
tokens_by_freq = user_reviews.filter(lambda x: (x.Translated_Review != 'nan') & (x.Translated_Review is not None)).distinct()\
.map(lambda x: x.Translated_Review).flatMap(lambda x: word_tokenize(x))\
.map(lambda x: (x.lower(), 1)).reduceByKey(lambda x,y: x+y).cache()

Obtengo los tokens más frecuentes ordenando por sus respectivas frecuencias. Podría haber hecho simplemente un takeOrdered( 30, -x[1] ), pero de esta manera obtenía los tokens con sus frecuencias y no me va a servir para lo que quiero hacer más adelante.

In [ ]:
most_frequent_tokens = tokens_by_freq.sortBy(lambda x: -x[1]).map(lambda x: x[0]).take(30)
#puedo hacer un map para "dar vuelta" clave-valor y luego hacer sortByKey (en vez de sortBy)

In [ ]:
most_frequent_tokens

['.',
 'i',
 ',',
 '!',
 'it',
 'game',
 'the',
 "'s",
 'good',
 'like',
 'app',
 'this',
 'great',
 'love',
 'get',
 'time',
 '...',
 "n't",
 '?',
 'would',
 'really',
 'even',
 'not',
 'ca',
 'update',
 'phone',
 'you',
 'work',
 "'m",
 'please']

# !!!!!!
Podría también haber paralelizado most_frequent_tokens y luego filtrar ese rdd por stopwords (no habría hecho falta cachear anteriormente), no sé verdaderamente qué es mejor.


Obtengo, inevitablemente, los 30 tokens más frecuentes en una lista. Puedo filtrar el rdd de tokens (el que tengo cacheado) con esta lista y la lista de stopwords.

El map lo hago para mostrar únicamente los tokens, no es necesario si no importa ver la frecuencia de cada token en las reviews.

Puedo hacer un collect ya que se que tengo un conjunto acotado de palabras.

In [ ]:
stopwords = set(stopwords.words('english'))

In [ ]:
tokens_by_freq.filter(lambda x: (x[0] in most_frequent_tokens) &  (x[0] in stopwords)).map(lambda x: x[0]).collect()

['i', 'this', 'it', 'not', 'you', 'the']

Por "culpa" de las abreviaciones nos quedan separados "n't" de "not", nos queda "'m" en vez de "am", "'s" en vez de "is" y algo similar sucede con "ca", que probablemente queda así porque el tokenizer separa "can't" literalmente en dos, en vez de considerar que realmente es "cannot".

Tanto am, is como can están entre las stopwords pero por aparecer abreviadas no se tienen en cuenta.

Not debería tener mayor frecuencia juntado los not y los n't.

Podría haber eliminado símbolos (como el punto o la coma, que quedaron entre los primeros) de las reviews para obtener resultados solo con palabras, pero no sé si era la idea del ejercicio ya que estos pueden ser considerados como tokens.